# Capstone Project: Predicting additional tax revenue for the state of Connecticut if sports wagering is legalized

## Problem Statement

The State Legislature of Connecticut is reviewing a bill to legalize sports wagering within the state. The State Legislature must carefully weigh pros and cons when deciding pass or reject the bill. If they pass the bill, the bill moves on to the Sentate for final approval. If the Senate approves, Connecticut would become the 12th state in the United States where sports wagering has been legalized.

A major consideration by the State Legislature is the potential revenue gain in the State General fund. All sources of gaming share revenue with the state and make transfers into the State General fund at the end of the fiscal year. While sports wagering itsself is likely to bring in new revenue for the state, the state must consider reductions in other revenue streams that the new sports wagering revenue stream may cause. 

Using Sports Wagering year-end reports we have for each state where sports wagering was legal in 2019 as well as state annual reports showing other streams of revenue, we will build and leverage an unsupervised linear regression model that will help predict if legazing sports wagering will lead to an increase or decrease in total revenue for Connectiut, using the other states as prototypes. After modeling, We will exmaine the model's mean squared error, helping us determine the success of the model. Our goal is to provide the state with a model that can predict revenue as closely as possible, giving the state more confidence to make their decision more clear.

In [14]:
import numpy as np
import pandas as pd
import time
import datetime as dt
import requests
import matplotlib.pyplot as plt
import regex as re
import seaborn as sns
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.naive_bayes import BernoulliNB, MultinomialNB

In [15]:
df = pd.read_csv('./data/state_reports.csv')

In [16]:
df

,state,month_year,handle,revenue,hold,taxes
0,NJ,19-Jan,"$385,279,662","$18,777,582",4.90%,"$2,532,619"
1,NaN,19-Feb,"$320,368,087","$12,732,740",4.00%,"$1,817,553"
2,NaN,19-Mar,"$372,451,342","$31,669,387",8.50%,"$4,180,051"
3,NaN,19-Apr,"$313,719,562","$21,215,747",6.80%,"$2,817,206"
4,NaN,19-May,"$318,940,677","$15,536,384",4.90%,"$2,135,704"
...,...,...,...,...,...,...
147,state,month_year,handle,revenue,hold,taxes
148,OR,19-Oct,"$5,603,136","$218,107",3.90%,"$196,296"
149,NaN,19-Nov,"$17,089,383","$943,687",5.50%,"$849,318"
150,NaN,19-Dec,"$22,375,049","$1,632,412",7.30%,"$1,469,171"


## Cleaning / EDA

In [22]:
df.info

<bound method DataFrame.info of      state  month_year           handle        revenue   hold         taxes
0       NJ      19-Jan    $385,279,662    $18,777,582   4.90%   $2,532,619 
1      NaN      19-Feb    $320,368,087    $12,732,740   4.00%   $1,817,553 
2      NaN      19-Mar    $372,451,342    $31,669,387   8.50%   $4,180,051 
3      NaN      19-Apr    $313,719,562    $21,215,747   6.80%   $2,817,206 
4      NaN      19-May    $318,940,677    $15,536,384   4.90%   $2,135,704 
..     ...         ...              ...            ...    ...           ...
147  state  month_year           handle        revenue   hold         taxes
148     OR      19-Oct      $5,603,136       $218,107   3.90%     $196,296 
149    NaN      19-Nov     $17,089,383       $943,687   5.50%     $849,318 
150    NaN      19-Dec     $22,375,049     $1,632,412   7.30%   $1,469,171 
151    NaN    Total-19  $1,557,191,900   $151,789,939   5.57%  $15,964,888 

[152 rows x 6 columns]>

In [23]:
df.columns

Index(['state', 'month_year', 'handle', 'revenue', 'hold', 'taxes'], dtype='object')

In [24]:
df.dtypes

state         object
month_year    object
handle        object
revenue       object
hold          object
taxes         object
dtype: object

In [25]:
df.shape

(152, 6)

In [26]:
df.count

<bound method DataFrame.count of      state  month_year           handle        revenue   hold         taxes
0       NJ      19-Jan    $385,279,662    $18,777,582   4.90%   $2,532,619 
1      NaN      19-Feb    $320,368,087    $12,732,740   4.00%   $1,817,553 
2      NaN      19-Mar    $372,451,342    $31,669,387   8.50%   $4,180,051 
3      NaN      19-Apr    $313,719,562    $21,215,747   6.80%   $2,817,206 
4      NaN      19-May    $318,940,677    $15,536,384   4.90%   $2,135,704 
..     ...         ...              ...            ...    ...           ...
147  state  month_year           handle        revenue   hold         taxes
148     OR      19-Oct      $5,603,136       $218,107   3.90%     $196,296 
149    NaN      19-Nov     $17,089,383       $943,687   5.50%     $849,318 
150    NaN      19-Dec     $22,375,049     $1,632,412   7.30%   $1,469,171 
151    NaN    Total-19  $1,557,191,900   $151,789,939   5.57%  $15,964,888 

[152 rows x 6 columns]>

In [28]:
df.isnull().sum().sort_values(ascending = False)

state         131
taxes          30
hold           30
revenue        30
handle         30
month_year     30
dtype: int64

In [29]:
df.fillna(' ', inplace = True)

In [30]:
df.isnull().sum().sort_values()

state         0
month_year    0
handle        0
revenue       0
hold          0
taxes         0
dtype: int64

In [32]:
df.head()

,state,month_year,handle,revenue,hold,taxes
0,NJ,19-Jan,"$385,279,662","$18,777,582",4.90%,"$2,532,619"
1,,19-Feb,"$320,368,087","$12,732,740",4.00%,"$1,817,553"
2,,19-Mar,"$372,451,342","$31,669,387",8.50%,"$4,180,051"
3,,19-Apr,"$313,719,562","$21,215,747",6.80%,"$2,817,206"
4,,19-May,"$318,940,677","$15,536,384",4.90%,"$2,135,704"


In [33]:
df.describe

<bound method NDFrame.describe of      state  month_year           handle        revenue   hold         taxes
0       NJ      19-Jan    $385,279,662    $18,777,582   4.90%   $2,532,619 
1               19-Feb    $320,368,087    $12,732,740   4.00%   $1,817,553 
2               19-Mar    $372,451,342    $31,669,387   8.50%   $4,180,051 
3               19-Apr    $313,719,562    $21,215,747   6.80%   $2,817,206 
4               19-May    $318,940,677    $15,536,384   4.90%   $2,135,704 
..     ...         ...              ...            ...    ...           ...
147  state  month_year           handle        revenue   hold         taxes
148     OR      19-Oct      $5,603,136       $218,107   3.90%     $196,296 
149             19-Nov     $17,089,383       $943,687   5.50%     $849,318 
150             19-Dec     $22,375,049     $1,632,412   7.30%   $1,469,171 
151           Total-19  $1,557,191,900   $151,789,939   5.57%  $15,964,888 

[152 rows x 6 columns]>